In [1]:
% matplotlib inline

from scikits.talkbox.features import mfcc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pyaudio as pa
import re
import os
from collections import defaultdict
from collections import Counter

random_state=24

音声ファイルimport

In [2]:
files = []
datapath = '../cry_data/devided_wav_data/'

for x in os.listdir(datapath):  
    if '.wav' in x:
        files.append(x)

mfccで特徴量とラベルのセット作成

In [3]:
label_dict = {'hu':0, 'ti':1, 'dc':2}
X_dict = defaultdict(list)
y_dict = defaultdict(list)
for f in files:
    x, sample_rate = sf.read(datapath + f)
    x = np.clip(x, 1e-10, 1)
    ceps,mspec,spec = mfcc(x, nwin=256, nfft=512, fs=8000, nceps=13)
    X_dict[re.sub(r'^\d*-', '', f)].append(np.mean(ceps, axis=0))
    if '-hu.' in f:
        y_dict[re.sub(r'^\d*-', '', f)].append(label_dict['hu'])
    elif '-ti.' in f:
        y_dict[re.sub(r'^\d*-', '', f)].append(label_dict['ti'])
    else:
        y_dict[re.sub(r'^\d*-', '', f)].append(label_dict['dc'])
X_dict = dict(X_dict)
y_dict = dict(y_dict)
for k, v in X_dict.items():
    X_dict[k] = np.array(v)
for k, v in y_dict.items():
    y_dict[k] = np.array(v)

分類器で学習、評価

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

In [9]:
precision = 0
recall = 0
f_value = 0

n_splits = 3

skf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
file_list = np.array(list(X_dict.keys()))
X = np.array(list(X_dict.values()))
y = np.array(list(y_dict.values()))
y_sample = np.array([a[0] for a in y])
for train_index, test_index in skf.split(file_list, y_sample):
    X_t, X_v = X[train_index], X[test_index]
    y_t, y_v = y[train_index], y[test_index]
    X_train = []
    y_train = []
    for v1 in X_t:
        for v2 in v1:
            X_train.append(v2)
    X_train = np.array(X_train)
    for v1 in y_t:
        for v2 in v1:
            y_train.append(v2)
    y_train = np.array(y_train)
    
    #clf = LinearSVC(random_state=random_state)
    clf = RandomForestClassifier(n_estimators=498, random_state=random_state)
    # clf = XGBClassifier(max_depth=8, learning_rate=0.05, n_estimators=700, seed=random_state)
    clf.fit(X_train, y_train)
    
    pred_array = []
    for i in range(len(X_v)):
        pred = clf.predict(X_v[i])
        pred_array.append(Counter(pred).most_common(1)[0][0])
    pred_array = np.array(pred_array)
    y_valid = np.array([a[0] for a in y_v])
    
    precision += precision_score(y_valid, pred_array, average='macro')
    recall += recall_score(y_valid, pred_array, average='macro')
    f_value += f1_score(y_valid, pred_array, average='macro')

precision = precision / float(n_splits)
recall = recall / float(n_splits)
f_value = f_value / float(n_splits)

/home/kakiuchi/.pyenv/versions/2.7.13/envs/jphacks2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/kakiuchi/.pyenv/versions/2.7.13/envs/jphacks2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
f_value

0.34539842873176202

In [66]:
pred_array

array([0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2])